# Example: Memory Representation of Floating Point Numbers
This example will familiarize students with the memory layout of floating point numbers. Floating point numbers $x\in\mathbb{R}$ are stored using 4$\times$bytes (single-precision) or 8$\times$bytes (double-precision)
following the [IEEE-754 standard](https://en.wikipedia.org/wiki/IEEE_754). 
The components of number $x\in\mathbb{R}$ are encoded in different segments of the`32-bit` or `64-bit` word, e.g., for a `64-bit` Float:
\begin{equation*}
x = -1^{S}\times{M}\times{2}^{(E-1023)}
\end{equation*}
where $S$ denotes the sign bit, $M$ denotes the mantissa (fraction) and $E$ denotes the exponent.
* For a `32-bit` floating point number, $S$ is bit 31 denoted by $a_{31}$, $M$ is encoded in bits $a_0\rightarrow{a_{22}}$ and $E$ is encoded by bits $a_{23}\rightarrow{a_{30}}$.
* For a `64-bit` floating point number, the sign bit $S$ is $a_{63}$, the mantissa $M$ is the number encoded by bits $a_0\rightarrow{a_{51}}$, and the exponent $E$ is encoded by bits $a_{52}\rightarrow{a_{62}}$.

## Task 1: Compute the bit pattern for a Floating point number
Let's specify a `64-bit` floating point number in the variable `floating_point_nunber_example` and then reconstruct that number from the bit pattern:

In [3]:
floating_point_nunber_example = 2.311898736;

We know we can get the bit pattern by calling [the `bitstring(...)` method](https://docs.julialang.org/en/v1/base/numbers/#Base.bitstring), but the wrinkle is that [`bitstring(...)`](https://docs.julialang.org/en/v1/base/numbers/#Base.bitstring) returns the bit pattern as [a `String` type](https://docs.julialang.org/en/v1/manual/strings/#man-strings). We'll have to convert [the `String`](https://docs.julialang.org/en/v1/manual/strings/#man-strings) to an array of `0` and `1`, but more later.

In [5]:
bitstring(floating_point_nunber_example)

"0100000000000010011111101100010011000011101101100100010100011000"

To understand where to go from here, we need a few things. The following logic contains a few advanced things, e.g., working with arrays and [`String` and `Char` types](https://docs.julialang.org/en/v1/manual/strings/#man-strings), function [piping `|>`](https://docs.julialang.org/en/v1/manual/functions/#Function-composition-and-piping), etc; don't worry too much about these now:
* We can convert a string into an array, i.e., an ordered list of [`Char` types](https://docs.julialang.org/en/v1/manual/strings/#man-characters) using the [`collect(...)` method](https://docs.julialang.org/en/v1/base/collections/#Base.collect-Tuple{Type,%20Any}). If we have the [`Char`](https://docs.julialang.org/en/v1/manual/strings/#man-characters) version of `1` or `0`, we can then try to convert it into an `Int` using the [`parse(...)` method](https://docs.julialang.org/en/v1/base/numbers/#Base.parse) in Julia.
* _Aside_: in older languages such as `C,` strings were natively represented as arrays of characters; in fact, there was no formal string class in `C.` However, newer languages that include [Unicode character sets](https://en.wikipedia.org/wiki/Unicode) have dedicated [`String` types](https://docs.julialang.org/en/v1/manual/strings/#man-strings) that are more complex. 
* We daisy chain together commands, i.e., `cmd_1 |> cmd_2` where the output of `cmd_1` is the input into `cmd_2` using the [Julia `|>` piping operator](https://docs.julialang.org/en/v1/manual/functions/#Function-composition-and-piping)

In [7]:
bit_pattern_array = bitstring(floating_point_nunber_example) |> collect |> reverse .|> x-> parse(Int,x)

64-element Vector{Int64}:
 0
 0
 0
 1
 1
 0
 0
 0
 1
 0
 1
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 1
 0

`Hack`: fix the `1`-based array issue by copying the `bit_pattern_array` into a dictionary (which we can make `0`-based), called `bit_patten_dictionary::Dict{Int64,Int64}`:

In [9]:
bit_patten_dictionary = Dict{Int64,Int64}(); # what are we doing here?
for i ∈ eachindex(bit_pattern_array)
    bit_patten_dictionary[i-1] = bit_pattern_array[i] # what are we doing here?
end
bit_patten_dictionary; # what is going on here?

### Why can't we use the digits method?
We have previously used [the `digits(...)` method](https://docs.julialang.org/en/v1/base/numbers/#Base.digits) to produce the bit pattern for [Integer numbers](https://docs.julialang.org/en/v1/manual/integers-and-floating-point-numbers/#Integers), can we do the same for [Floating point numbers](https://docs.julialang.org/en/v1/manual/integers-and-floating-point-numbers/#Floating-Point-Numbers)?

In [11]:
digits(floating_point_nunber_example, base=10, pad=64) # try to compute pattern for floating_point_nunber_example

LoadError: MethodError: no method matching digits(::Float64; base::Int64, pad::Int64)

[0mClosest candidates are:
[0m  digits([91m::Type{<:Integer}[39m, [91m::Integer[39m; base, pad)
[0m[90m   @[39m [90mBase[39m [90m[4mintfuncs.jl:977[24m[39m
[0m  digits([91m::Integer[39m; base, pad)
[0m[90m   @[39m [90mBase[39m [90m[4mintfuncs.jl:974[24m[39m


## Task 2: Compute the sign bit `S`
The `sign` bit for a `64-bit` number is in position `63`, i.e., $a_{63}$. Let's look up that value in the `bit_patten_dictionary` and set it equal to the variable `S.` 
* We need to compute the sign manually because of the weirdness about zero exponents on negative numbers; store this value in the `sign` variable. The default value `sign = 1` (positive). However, if the `sign` bit is `1`, i.e., $a_{63} = 1$, we set `sign = -1` (negative).

In [60]:
sign = 1;
S = bit_patten_dictionary[63]
if (S == 1)
    sign = -1;
end
println("The sign bit equals sign = $(sign)") # what?

The sign bit equals sign = 1


## Task 3: Compute the Mantissa `M`
For a `64-bit` floating point number, the mantissa `M` is stored in bits `0` $\rightarrow$ `51`. However, we run the sum from `1` $\rightarrow$ `52` because of numbering weirdness. Let's create a collection holding this range and store it in the `mantissa_bit_range::Array{Int64,1}` variable

In [39]:
mantissa_bit_range = range(1,stop=52,step = 1) |> collect

52-element Vector{Int64}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
  ⋮
 41
 42
 43
 44
 45
 46
 47
 48
 49
 50
 51
 52

Implement the expression from the notes to compute the mantissa `M.` For a `64-bit` word size, the `M` expression is given by:
\begin{equation*}
M = \left(1+\sum_{i=1}^{52}a_{52-i}\cdot{2}^{-i}\right)
\end{equation*}
Iterate over each index of the `mantissa_bit_range` collection using [the `eachindex(...)` method](https://docs.julialang.org/en/v1/base/arrays/#Base.eachindex), access the value from the `mantissa_bit_range` and store it in the variable `i`, access the $a_{\star}$ value from the `bit_patten_dictionary` and store it in the `bₖ` variable, and then sequentially compute the `M` value.
* __Wait__: What happened to the `+1` value? In the implementation below, we don't have a `+1` leading term. Check out [the documentation on Mathematical operators to see what is going on with the `+=` operation](https://docs.julialang.org/en/v1/manual/mathematical-operations/#Mathematical-Operations-and-Elementary-Functions).

In [89]:
M = 1.0;
base = 2.0;
for k ∈ eachindex(mantissa_bit_range)
    i = mantissa_bit_range[k]
    aₖ = bit_patten_dictionary[52-i]
    M += aₖ*(base^(-i))
end
println("The Mantissa M = $(M)")

The Mantissa M = 1.155949368


## Task 4: Compute the exponent `E`
For a `64-bit` floating point number, the exponent `E` is stored in bits `52` $\rightarrow$ `62`. Let's create a collection holding this range and store it in the `exponent_bit_range::Array{Int64,1}` variable using [the `range(...)` method](https://docs.julialang.org/en/v1/base/math/#Base.range) in combination with [the `collect(...)` method](https://docs.julialang.org/en/v1/base/collections/#Base.collect-Tuple{Type,%20Any}).

In [65]:
exponent_bit_range = range(52,stop=62, step = 1) |> collect

11-element Vector{Int64}:
 52
 53
 54
 55
 56
 57
 58
 59
 60
 61
 62

Now, let's write some code to compute the exponent `E.` The exponent `E` is the `base 10` realization of the binary number $(a_{62},\dots,a_{52})_{2}$. Thus, we compute this number assuming `base = 2`, where we access the bit values from the `bit_patten_dictionary` dictionary (save these in the `bₖ` variable).
* Why $\text{base}^{k-1}$? The `exponent_bit_range` is `1`-based, so we need to subtract `1` to make it `0`-based.

In [81]:
E = 0.0
base = 2;
for k ∈ eachindex(exponent_bit_range)   
    index = exponent_bit_range[k]
    aₖ = bit_patten_dictionary[index]    
    E += aₖ*(base^(k-1))
end
println("The exponent E = $(E)")

The exponent E = 1024.0


## Task 5: Putting everything together
Finally, now that we have computed the `S,` `M,` and `E` components, let's combine them and see if we recover the original value specified in the `floating_point_nunber_example` variable.

In [22]:
x = (sign)*M*(2^(E - 1023))

2.311898736

Use the [Julia @assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) in combination with [the `isapprox (...)` method](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) to test the equality of the original floating point number and our reconstructed version of that number.
* Why `isapprox`? The [`isapprox(...)` method](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) encodes an inexact equality comparison, i.e., we can start to think about the precision of a floating point number and compare the values at different levels of precision using either the absolute tolerance (specified by the `atol` keyword argument) or the relative tolerance (defined by the `rtol` keyword argument). 

In [106]:
@assert isapprox(x, floating_point_nunber_example, atol=1e-12)